In [ ]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time

In [38]:
def crawl(url, depth,visited):
    if depth < 2 :
        visited.add(url)
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
        time.sleep(0.35)
        response = requests.get(url,headers=headers)
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
        except:
            soup = BeautifulSoup(response.text, 'lxml')
        links = soup.find_all('a')
        links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
        links = [urljoin(url, link) for link in links if link]

        for link in links:
            if link not in visited:
                visited.add(link)
                #if link.startswith(url):
                crawl(link,depth=depth+1, visited=visited)
    return visited
base_url = 'https://news.kapook.com'
website_dict3 = crawl(base_url, 0, set())
print(website_dict3)

{'https://covid-19.kapook.com/ https:/covid-19.kapook.com/view222266.html', 'https://creator.kapook.com/view264836.html', 'https://hilight.kapook.com/view/230882', 'https://football.kapook.com/newslist.php?page=4', 'https://news.kapook.com/topics/มหาวิทยาลัย  ', 'https://movie.kapook.com/view260266.html', 'https://www.coolism.net', 'https://travel.kapook.com/view130190.html', 'https://pet.kapook.com/view215176.html', 'https://news.kapook.com/topics/%E0%B8%AB%E0%B8%A5%E0%B8%B1%E0%B8%81%E0%B8%AA%E0%B8%B9%E0%B8%95%E0%B8%A3%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%A8%E0%B8%B6%E0%B8%81%E0%B8%A9%E0%B8%B2', 'https://horoscope.kapook.com/tarot.php ', 'https://hilight.kapook.com/view/189222', 'http://news.kapook.com/topics/เด็กพิเศษ', 'https://hilight.kapook.com/view/230699', 'https://hilight.kapook.com/view/78632 ', 'http://women.kapook.com/view265565.html', 'https://www.orient-thai.com', 'https://movie.kapook.com/view251071.html', 'https://cooking.kapook.com/view196919.html', 'https://movie.kapook.co

In [39]:
def check_domain(base_url,links):
    result= set()
    for link in links :
        if link.startswith(base_url):
            result.add(link)
    return result

check_domain(base_url,website_dict3)

{'https://news.kapook.com',
 'https://news.kapook.com ',
 'https://news.kapook.com  ',
 'https://news.kapook.com/',
 'https://news.kapook.com/ https:/health.kapook.com/view262704.html',
 'https://news.kapook.com/accident',
 'https://news.kapook.com/chinese',
 'https://news.kapook.com/crime',
 'https://news.kapook.com/crime ',
 'https://news.kapook.com/drug',
 'https://news.kapook.com/it',
 'https://news.kapook.com/other',
 'https://news.kapook.com/politic',
 'https://news.kapook.com/region',
 'https://news.kapook.com/royal',
 'https://news.kapook.com/science',
 'https://news.kapook.com/social_media',
 'https://news.kapook.com/social_problem',
 'https://news.kapook.com/south',
 'https://news.kapook.com/sport',
 'https://news.kapook.com/sport ',
 'https://news.kapook.com/topics/',
 'https://news.kapook.com/topics/ https:/health.kapook.com/view262704.html',
 'https://news.kapook.com/topics/%E0%B8%81%E0%B8%A3%E0%B8%B2%E0%B8%94%E0%B8%A2%E0%B8%B4%E0%B8%87',
 'https://news.kapook.com/topics/%

In [40]:
def check_not_domain(base_url,links):
    result= set()
    for link in links :
        if not link.startswith(base_url):
            result.add(link)
    return result

not_domain = check_not_domain(base_url,website_dict3)
print(not_domain)

{'https://covid-19.kapook.com/ https:/covid-19.kapook.com/view222266.html', 'https://creator.kapook.com/view264836.html', 'https://hilight.kapook.com/view/230882', 'https://football.kapook.com/newslist.php?page=4', 'https://movie.kapook.com/view260266.html', 'https://www.coolism.net', 'https://travel.kapook.com/view130190.html', 'https://pet.kapook.com/view215176.html', 'https://horoscope.kapook.com/tarot.php ', 'https://hilight.kapook.com/view/189222', 'http://news.kapook.com/topics/เด็กพิเศษ', 'https://hilight.kapook.com/view/230699', 'https://hilight.kapook.com/view/78632 ', 'http://women.kapook.com/view265565.html', 'https://www.orient-thai.com', 'https://movie.kapook.com/view251071.html', 'https://movie.kapook.com/view257796.html', 'https://cooking.kapook.com/view196919.html', 'https://women.kapook.com/view219140.html', 'https://pet.kapook.com/news', 'https://covid-19.kapook.com/relate', 'https://hilight.kapook.com/view/230351', 'https://www.komchadluek.net', 'https://hilight.kapo

In [41]:
target_links=['https://www.thairath.co.th','https://news.kapook.com']
def check_ref(base,links):
    base_links=dict()
    for i in base:
        base_links[i] = 0
    for i in links:
        for j in base:
            if i.startswith(j):
                base_links[j]+=1
    return base_links

ref=check_ref(target_links,not_domain)
print(ref)
                

{'https://www.thairath.co.th': 2, 'https://news.kapook.com': 0}


In [ ]:
class spyder:
    def __init__( self , base_url ):
        self.base_url = base_url
        self.target_links = target_links
    
    def get_crawler(self):
        self.result_crawler = self.crawl(self.base_url,0,set())
        return self.result_crawler
    
    def get_check_domain(self):
        self.base_url

    def crawl(self,base_url,depth,visited):
        if depth < 3 :
            visited.add(self.base_url)
            headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
            response = requests.get(self.base_url,headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            links = soup.find_all('a')
            links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
            links = [urljoin(self.base_url, link) for link in links if link]

            for link in links:
                if link not in visited:
                    visited.add(link)
                    crawl(link,depth=depth+1, visited=visited)
        return visited
    
    def check_domain(base_url,links):
        result= set()
        for link in links :
            if link.startswith(base_url):
                result.add(link)
        return result

In [ ]:
base_url = 'https://www.thairath.co.th'
web_spyder=spyder(base_url)
web_spyder.get_crawler()

In [ ]:
base_url = 'https://www.thairath.co.th'
target_links=['https://www.thairath.co.th','https://www.facebook.com']
website = crawl(base_url, 0, visited=set())
domain = check_domain(base_url,website)
not_domain = check_not_domain(base_url,website)
ref = check_ref(target_links,not_domain)
print(website)
print('-------------------------------------------')
print(domain)
print('-------------------------------------------')
print(not_domain)
print('-------------------------------------------')
print(ref)